In [35]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, InputLayer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [36]:
WINDOW_SIZE = 10
NUMBER_OF_FEATURES = 6
NUMBER_OF_CLASSES = 3
DATA_FOLDER = 'C:\Dev\\vr-kat-project-python-2\processed-training-data\\4-PROCESSED-DATA\TRAIN2\\'


ALL_X_TRAIN_CURRVEL_FILE_PATH = 'C:\\Dev\\vr-kat-project-python-2\\NeuralNetwork\\np-saved-data\\ALL-X-TRAIN-CURRVEL.npy'
ALL_Y_TRAIN_CURRVEL_FILE_PATH = 'C:\\Dev\\vr-kat-project-python-2\\NeuralNetwork\\np-saved-data\\ALL-Y-TRAIN-CURRVEL.npy'

ALL_X_TRAIN_ALLVEL_FILE_PATH = 'C:\\Dev\\vr-kat-project-python-2\\NeuralNetwork\\np-saved-data\\ALL-X-TRAIN-ALLVEL.npy'
ALL_Y_TRAIN_ALLVEL_FILE_PATH = 'C:\\Dev\\vr-kat-project-python-2\\NeuralNetwork\\np-saved-data\\ALL-Y-TRAIN-ALLVEL.npy'



In [38]:
file_names = [file for file in os.listdir(DATA_FOLDER) if file.endswith('.xlsx') and os.path.isfile(os.path.join(DATA_FOLDER, file))]

# TEMP LIMITER REMOVE WHEN READY
#file_names = [file_names[20]]

print(file_names)

['PROC-TRAIN2-SIDESTEPS-L-LAR-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-60BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-80BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-MED-110BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-MED-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-MED-220BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-MED-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-MED-55BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-MED-80BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-60BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-80BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-MED-110BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-MED-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-MED-220BPM-AUGMENT.xl

In [39]:
def normalizeSensorData(sensor_input):
    if(sensor_input < 0):
        return np.abs(sensor_input/180)
    else:
        return 0.5 + np.abs(sensor_input/180)

def normalizeXVelocity(velocity_input):
    if(velocity_input < 0):
        return np.abs(velocity_input/10)
    else:
        return 0.5 + np.abs(velocity_input/10)
    
def normalizeZVelocity(velocity_input):
    return np.abs(velocity_input/10)


In [40]:
def df2Xy(df, windowSize=5, fetchAllSpeed=False):
  X, y = [], []


  for i in range( len(df) - windowSize ):
    # inputs: X rows
    # form a new input which has size of our windowSize
    input = []

    # loop through each row in our windowsize
    for j in range(windowSize):
        # fetch sensor data for this row
        row_values = df.loc[i + j, ['L_Pitch', 'L_Roll', 'R_Pitch', 'R_Roll']].values.tolist()
        
        # If j is the last element, fetch X_Vel and Z_Vel
        if (j == windowSize - 1) or (fetchAllSpeed == True):
            row_values.extend(df.loc[i + j, ['X_Vel', 'Z_Vel']].values.tolist())
        else:
            # Fill columns 5 and 6 with zeros
            row_values.extend([0.0, 0.0])

        # add row values to the input
        input.append(row_values)

    # add our input to our total inputs, marked as X
    X.append(input)


    # outputs: y labels
    label = df.loc[i + windowSize, ['X_Vel_Positive', 'X_Vel_Negative', 'Z_Vel']].values.tolist()

    y.append(label)


  return (np.array(X), np.array(y))

In [49]:
ALL_X_TRAIN_CURRVEL = np.empty((0, WINDOW_SIZE, NUMBER_OF_FEATURES))  # List to store all X training data
ALL_Y_TRAIN_CURRVEL = np.empty((0, NUMBER_OF_CLASSES))  # List to store all Y training data

ALL_X_TRAIN_ALLVEL = np.empty((0, WINDOW_SIZE, NUMBER_OF_FEATURES))  # List to store all X training data
ALL_Y_TRAIN_ALLVEL = np.empty((0, NUMBER_OF_CLASSES))  # List to store all Y training data

In [52]:
def processData(ALL_X_TRAIN_CURRVEL, ALL_Y_TRAIN_CURRVEL, ALL_X_TRAIN_ALLVEL, ALL_Y_TRAIN_ALLVEL):
    for fileName in file_names:
        # Read the Excel file
        df = pd.read_excel(DATA_FOLDER + fileName)

        # NORMALIZE THE SENSOR DATA
        df["L_Pitch"] = df["L_Pitch"].apply(normalizeSensorData)
        df["L_Roll"] = df["L_Roll"].apply(normalizeSensorData)
        df["R_Pitch"] = df["R_Pitch"].apply(normalizeSensorData)
        df["R_Roll"] = df["R_Roll"].apply(normalizeSensorData)
        df["X_Vel"] = df["X_Vel"].apply(normalizeXVelocity)
        df["Z_Vel"] = df["Z_Vel"].apply(normalizeZVelocity)

        # CREATE NEW COLUMNS FOR Z VELOCITY (POSITIVE AND NEGATIVE)
        df['X_Vel_Positive'] = df['X_Vel'].mask(df['X_Vel'] < 0, 0)
        df['X_Vel_Negative'] = -1*df['X_Vel'].mask(df['X_Vel'] >= 0, 0)


        xTrain_CurrVel, yTrain_CurrVel = df2Xy(df, WINDOW_SIZE, False)
        xTrain_AllVel, yTrain_AllVel = df2Xy(df, WINDOW_SIZE, True)

        ALL_X_TRAIN_CURRVEL = np.concatenate([ALL_X_TRAIN_CURRVEL, xTrain_CurrVel])
        ALL_Y_TRAIN_CURRVEL = np.concatenate([ALL_Y_TRAIN_CURRVEL, yTrain_CurrVel])

        ALL_X_TRAIN_ALLVEL = np.concatenate([ALL_X_TRAIN_ALLVEL, xTrain_AllVel])
        ALL_Y_TRAIN_ALLVEL = np.concatenate([ALL_Y_TRAIN_ALLVEL, yTrain_AllVel])

        
        print('xTrain_CurrVel.shape:', xTrain_CurrVel.shape, ' xTrain_CurrVel.shape:', xTrain_CurrVel.shape)
        print('ALL_X_TRAIN_CURRVEL.shape:', ALL_X_TRAIN_CURRVEL.shape, ' ALL_Y_TRAIN_CURRVEL.shape:', ALL_Y_TRAIN_CURRVEL.shape)
        print('-------------------------------')

    # Assuming your numpy array is called 'data_array'
    np.save(ALL_X_TRAIN_CURRVEL_FILE_PATH, ALL_X_TRAIN_CURRVEL)
    np.save(ALL_Y_TRAIN_CURRVEL_FILE_PATH, ALL_Y_TRAIN_CURRVEL)
    np.save(ALL_X_TRAIN_ALLVEL_FILE_PATH, ALL_X_TRAIN_ALLVEL)
    np.save(ALL_Y_TRAIN_ALLVEL_FILE_PATH, ALL_Y_TRAIN_ALLVEL)


In [51]:
if os.path.exists(ALL_X_TRAIN_CURRVEL_FILE_PATH):
    ALL_X_TRAIN_CURRVEL = data_array = np.load(ALL_X_TRAIN_CURRVEL_FILE_PATH)
    ALL_Y_TRAIN_CURRVEL = data_array = np.load(ALL_Y_TRAIN_CURRVEL_FILE_PATH)
    ALL_X_TRAIN_ALLVEL = data_array = np.load(ALL_X_TRAIN_ALLVEL_FILE_PATH)
    ALL_Y_TRAIN_ALLVEL = data_array = np.load(ALL_Y_TRAIN_ALLVEL_FILE_PATH)
else:
    processData(ALL_X_TRAIN_CURRVEL, ALL_Y_TRAIN_CURRVEL, ALL_X_TRAIN_ALLVEL, ALL_Y_TRAIN_ALLVEL)


UnboundLocalError: cannot access local variable 'ALL_X_TRAIN_CURRVEL' where it is not associated with a value

In [45]:
# Generate random permutation indices
random_indices_1 = np.random.permutation(len(ALL_X_TRAIN_CURRVEL))

# Shuffle the input features and target labels using the random indices
shuffled_X_TRAIN_CURR_VEL = ALL_X_TRAIN_CURRVEL[random_indices_1]
shuffled_Y_TRAIN_CURR_VEL = ALL_Y_TRAIN_CURRVEL[random_indices_1]

shuffled_X_TRAIN_ALL_VEL = ALL_X_TRAIN_ALLVEL[random_indices_1]
shuffled_Y_TRAIN_ALL_VEL = ALL_Y_TRAIN_ALLVEL[random_indices_1]

print(shuffled_X_TRAIN_CURR_VEL)

[]


In [ ]:

shuffled_X_TRAIN_ALL_VEL = ALL_X_TRAIN_ALLVEL[random_indices_2]
shuffled_Y_TRAIN_ALL_VEL = ALL_Y_TRAIN_ALLVEL[random_indices_2]

In [14]:
model2 = Sequential()
model2.add(InputLayer((WINDOW_SIZE, NUMBER_OF_FEATURES)))
model2.add(LSTM(128))
model2.add(Dense(16, 'relu'))
model2.add(Dense(8, 'relu'))
model2.add(Dense(NUMBER_OF_CLASSES, 'relu'))

model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               68096     
                                                                 
 dense (Dense)               (None, 16)                2064      
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 3)                 27        
                                                                 
Total params: 70,323
Trainable params: 70,323
Non-trainable params: 0
_________________________________________________________________


In [47]:
cp2 = ModelCheckpoint('model2/', save_best_only=True)
model2.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=['acc']) #metrics=[RootMeanSquaredError()])

In [48]:
history2 = model2.fit(shuffled_X_TRAIN, shuffled_Y_TRAIN, epochs=50, callbacks=[cp2])

Epoch 1/50
4052/4052 [==============================] - 35s 8ms/step - loss: 0.1995 - acc: 0.9480
Epoch 2/50
4052/4052 [==============================] - 37s 9ms/step - loss: 0.1902 - acc: 0.9583
Epoch 3/50
4052/4052 [==============================] - 37s 9ms/step - loss: 0.1890 - acc: 0.9625
Epoch 4/50
4052/4052 [==============================] - 37s 9ms/step - loss: 0.1885 - acc: 0.9625
Epoch 5/50
4052/4052 [==============================] - 38s 9ms/step - loss: 0.1881 - acc: 0.9628
Epoch 6/50
4052/4052 [==============================] - 34s 8ms/step - loss: 0.1874 - acc: 0.9637
Epoch 7/50
4052/4052 [==============================] - 37s 9ms/step - loss: 0.1845 - acc: 0.9615
Epoch 8/50
4052/4052 [==============================] - 38s 9ms/step - loss: 0.1788 - acc: 0.9629
Epoch 9/50
4052/4052 [==============================] - 41s 10ms/step - loss: 0.1767 - acc: 0.9624
Epoch 10/50
4052/4052 [==============================] - 40s 10ms/step - loss: 0.1750 - acc: 0.9626
Epoch 11/50
4052/